### 손글씨 데이터로 구현하는 신경망의 순전파(forward propagation)
- 배치 처리 및 배열을 이용한 처리

In [39]:
import sys, os
sys.path.append(os.pardir)
from dataset.mnist import load_mnist

(x_train,t_train),(x_test,t_test) = load_mnist(flatten=True,normalize=False)
# flatten : 데이터셋을 평탄화(1차원 배열) 하는 작업 1x28x28 -> 784짜리 배열
# normalize : 모두 0과 1 사이의 값으로 정규화

In [28]:
import sys, os
sys.path.append(os.pardir)
from dataset.mnist import load_mnist
import numpy as np
from PIL import Image

def img_show(img):
    pill_img = Image.fromarray(np.uint8(img))
# https://bskyvision.com/736
# np의 uint8은 부호없는 정수 자료형으로 2^8개 총 256개의 정수를 표현할 수 있다
    pill_img.show()

(x_train,t_train),(x_test,t_test) = load_mnist(flatten=True,normalize=False)
# flatten : 데이터셋을 평탄화(1차원 배열) 하는 작업 1x28x28 -> 784짜리 배열
# normalize : 모두 0과 1 사이의 값으로 정규화

img = x_train[0]
# 이미지의 픽셀데이터
img = img.reshape(28,28)
label = t_train[0]
print(label); img_show(img)
# 픽셀데이터를 28x28로 바꿔야 이미지가 정상적으로 출력이된다.


5


손글씨 데이터 (784->50->100->10)인 신경망 추론처리
- get_data() : 추론처리할 test 데이터 불러오기
- init_network() : pickle 파일 이용해서 학습된 가중치 매개변수 읽어오기
- predict() : 가중치 매개변수를 통해 추론해보기

In [67]:
# 활성화 함수 목록
def sigmoid(x):
    return 1/(1+np.exp(-x))

def softmax(x):
    c = np.max(x)
    exp_x = np.exp(x-c)
    return exp_x / np.sum(exp_x)

In [72]:
import sys, os
import pickle
sys.path.append(os.pardir)
def get_data():
    (x_train,t_train),(x_test,t_test) = load_mnist(flatten=True,normalize=False)
    return x_test,t_test
def init_network():
    with open("sample_weight.pkl","rb") as  f:
        network = pickle.load(f)
    return network

def predict(network,x):
    W1,W2,W3 = network["W1"],network["W2"],network["W3"]
    b1,b2,b3 = network["b1"],network["b2"],network["b3"]
    A1 = np.dot(x,W1) + b1
    Z1 = sigmoid(A1)
    # 1번 은닉층을 지난 신호값
    
    A2 = np.dot(Z1,W2) + b2
    Z2 = sigmoid(A2)
    # 2번 은닉층을 지난 신호값
    
    A3 = np.dot(Z2,W3) + b3
    y = softmax(A3)
    return y

In [85]:
x,t = get_data()
network = init_network()
res = predict(network,x)
# 학습된 가중치를 통한 추론 결과
# x -> (10000,784) 총 10000개 데이터에 대한 분류 추론 결과를 얻을 수 있다
# res -> (10000,10) 총 10000개의 데이터가 어떤 숫자로 분류되는지...

<ipython-input-67-9bb87f033b0e>:3: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))


분류가 얼마나 정확한지 정확도 평가해보기
- t가 정답 label
- res가 추론 label

In [94]:
accuracy_cnt = 0
for i in range(res.shape[0]):
    y = res[i]
    p = np.argmax(y) # softmax로 분류되었으니까 확률이 가장 높은 인덱스 찾아내기
    if p == t[i]:
        accuracy_cnt +=1
        
print("Accuracy는 ",accuracy_cnt/res.shape[0])

Accuracy는  0.9207


정규화에 따른 성능 변화 확인
- 아래 버전은 정규화 O
- 위에 버전은 정규화 X
- 정규화 하게되면 성능 상승 OK

In [95]:
def get_data():
    (x_train,t_train),(x_test,t_test) = load_mnist(flatten=True,normalize=True)
    return x_test,t_test

x,t = get_data()
res = predict(init_network(),x)

accuracy_cnt = 0
for i in range(res.shape[0]):
    y = res[i]
    p = np.argmax(y) # softmax로 분류되었으니까 확률이 가장 높은 인덱스 찾아내기
    if p == t[i]:
        accuracy_cnt +=1
        
print("정규화한 Accuracy는 ",accuracy_cnt/res.shape[0])

정규화한 Accuracy는  0.9352


배치(Batch)처리를 통한 처리 시간 효율화
- 하나씩 계산하는 것보다 Batch 묶음으로 계산하는 것이 효율적
- Batch 100 사이즈

In [133]:
x,t = get_data()
network = init_network()

batch_size = 100 # 배치 크기
accuracy_cnt = 0

for i in range(0,len(x),batch_size):
    x_batch = x[i:i+batch_size]
    y_batch = predict(network,x_batch)
    # (100,10) 행렬 y_batch에 대해서 (axis=1) 1행 안에서 열을 기준으로 값이 최대인 것의 index를 찾아나간다
    p = np.argmax(y_batch,axis=1) # 총 100개의 행에 대해서 값이 가장 큰 열의 index를 다 모아놨다
    accuracy_cnt += np.sum(t[i:i+batch_size] == p)

In [134]:
print("배치처리하고, 정규화한 Accuracy는 ",accuracy_cnt/res.shape[0])

배치처리하고, 정규화한 Accuracy는  0.9352


배치 1000 사이즈로 바로 구현해보기

※ 총 10000개의 행에 대해서 예측된 값과 기존 값이 일치하는지 하나하나 비교할 필요는 없었다
- numpy 배열을 이용해서 한번에 처리가능했다

In [139]:
x,t = get_data()
network = init_network()
res = predict(network,x)

accuracy_cnt = np.sum(t == np.argmax(res,axis=1))
print("정규화한 Accuracy는 ",accuracy_cnt/res.shape[0])

정규화한 Accuracy는  0.9352


총평
- 신경망에서는 활성화 함수로 ReLU와 시그모이드와 같은 매끄러운 함수 사용함
- 출력층의 활성화 함수로 항등함수를 이용한다면 회귀, 소프트맥스를 이용하면 분류를 할 수 있다
- 분류에서는 출력층 뉴런(노드)의 수와 클래스(label)수와 같다
- 입력데이터를 묶은 것을 batch라 하며, 빠른 데이터 처리 계산이 가능하다